In [7]:
! pip3 install reddit_api_creds

Could not install packages due to an EnvironmentError: 404 Client Error: Not Found for url: https://pypi.org/simple/reddit-api-creds/



In [1]:
import praw
import pandas as pd
import requests
import datetime
import uuid
from reddit_api_creds import *

ModuleNotFoundError: No module named 'reddit_api_creds'

In [2]:

scraper = praw.Reddit(client_id='GENERTE_ONE',
                     client_secret='GENERTE_ONE',
                     password='password',
                     user_agent='testscript3',
                     username='NONE')

In [16]:
data_post=pd.DataFrame()
data_vote=pd.DataFrame()

In [17]:
# Kudos to Wikipedia https://en.wikipedia.org/wiki/Base36
def base36encode(integer: int) -> str:
    """Convert from Base10 to Base36."""
    chars = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    sign = '-' if integer < 0 else ''
    integer = abs(integer)
    result = ''

    while integer > 0:
        integer, remainder = divmod(integer, 36)
        result = chars[remainder] + result

    return sign + result

# Scraper

In [62]:
csuf = scraper.subreddit('bitcoin')

In [63]:
#scraped_posts = csuf.hot(limit=10000)
scraped_posts = csuf.top(limit=30)

In [64]:
data = []
data1 = []
for post in scraped_posts:
    # get base36 uuid
    uid = uuid.uuid1()
    uid = base36encode(uid.int)
    
    # username
    username = post.author
    if type(username) == praw.models.Redditor:
        username = post.author.name
    else:
        username = '[deleted]'
    
    # community_name
    community_name = post.subreddit.display_name
    data.append([uid, username, post.title, community_name, post.url, post.selftext, post.created])
    data1.append([uid, post.created ,post.score, community_name])

In [65]:
posts = pd.DataFrame(data, columns=['uuid', 'username', 'title', 'community_name', 'url', 'description', 'published'])
votes = pd.DataFrame(data1, columns=['uuid','published' ,'score', 'community_name'])
posts['published'] = [datetime.datetime.utcfromtimestamp(i).isoformat() for i in posts['published']]
# posts['username'] = [i.name if type(i)==praw.models.Redditor else '[deleted]' for i in posts['username']]
# posts['community_name'] = [i.display_name for i in posts['community_name']]
# votes['community_name'] = [i.display_name for i in votes['community_name']]
posts = posts.convert_dtypes()

# Shuffle dataframes

In [66]:
posts = posts.sample(frac=1, random_state=100).reset_index(drop=True)
votes = votes.sample(frac=1, random_state=100).reset_index(drop=True)

# Count and dtypes

In [67]:
print(posts.dtypes)

uuid              string
username          string
title             string
community_name    string
url               string
description       string
published         string
dtype: object


In [68]:
# print no of records in each community
posts.set_index(['uuid', 'community_name']).count(level='community_name')

,username,title,url,description,published
community_name,,,,,
Bitcoin,30,30,30,30,30


In [69]:
data_post=data_post.append(posts, ignore_index = True) 
data_vote=data_vote.append(votes, ignore_index = True) 

# Save to JSON and csv

In [70]:
data_post

,uuid,username,title,community_name,url,description,published
0,BOERRW8N26WEYZQSOVC7XUSJ,HotSwap_,Laid off for 8 weeks. Anyone else starting the...,Python,https://i.redd.it/uqia9wc853n41.jpg,,2020-03-17T03:23:03
1,BOERVITLEFY7M6DJVZ4YV8YR,R0dartha,I created a python script to generate color pa...,Python,https://i.redd.it/70kl5aweqpu31.jpg,,2019-10-26T00:21:00
2,BOERVY6NBND8V99VK3TIZ6EB,StreetStatistician,The entire MIT Intro Computer Science class us...,Python,https://www.youtube.com/watch?v=ytpJdnlu9ug&li...,,2018-12-21T03:43:12
3,BOERSQYQWLQHH5JG14PC5PNN,sothisiswhyhmm,"3 days ago, told my wife I’d learn to code and...",Python,https://i.redd.it/uchxh9rqnjq41.jpg,,2020-04-03T14:07:38
4,BOERR1IJ7S2CGTY5CLZ3PZXF,redditNewUser2017,I've made a 3D scanner that's fully automated ...,Python,https://v.redd.it/n8nsbsq0pfh41,,2020-02-17T15:12:31
...,...,...,...,...,...,...,...
145,AV6T9ONMCY9XNY11LK0OFPQG3,MichKOG,Almost everyone now is an Investor,Bitcoin,https://imgur.com/nb33jRX,,2017-12-01T02:58:39
146,AV6T9L8PXKMLDXDV06Z7YUAZN,benjaminikuta,Nothing can increase by that much and still be...,Bitcoin,https://i.imgur.com/oWePY7q.jpg,,2017-12-13T16:02:51
147,AV6T9KDZTQ7RBF82CUPUUMG4J,[deleted],Everyone who's trading BTC right now,Bitcoin,http://cdn.mutually.com/wp-content/uploads/201...,,2018-01-07T04:38:56
148,AV6T9OVAVWVN6KKHRE30PRP5V,swahlgren,Danish Bitcoin billionaire new sponsor of prof...,Bitcoin,https://www.business.dk/virksomheder/dansk-bit...,,2017-12-14T21:17:55


In [71]:
data_post.to_json('posts.json', index=False, orient='table', indent=1)

In [72]:
data_vote.to_json('votes.json', index=False, orient='table', indent=1)

In [73]:
data_post.to_csv('posts.csv')
data_vote.to_csv('votes.csv')